In [25]:
import deepchem as dc
import numpy as np
import random
import math
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import _LRScheduler
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm_

from collections import UserList, defaultdict
from tqdm import tqdm

from rdkit import RDLogger                                                                                                                                                               
RDLogger.DisableLog('rdApp.*') 

def seed_all():
    np.random.seed(42)
    random.seed(42)

In [26]:
seed_all()

#### Import dataset

In [27]:
# Load HIV dataset
hiv_tasks, datasets, transformers = dc.molnet.load_hiv(featurizer="ECFP", set="sparse", splitter = 'random')

In [28]:
# Split 80/10/10
train_dataset, valid_dataset, test_dataset = datasets
print(f'Compound train/valid/test split: {len(train_dataset)}/{len(valid_dataset)}/{len(test_dataset)}')

Compound train/valid/test split: 32901/4113/4113


In [29]:
# Select SMILES structures and label
train_data = list(train_dataset.ids)
train_label = list([int(l) for l in train_dataset.y])
print(f'Example SMILES: {train_data[0]}')
print(f'Example label: {train_label[0]}')

Example SMILES: O=C1C(=Cc2ccc(O)cc2)N=C(c2ccccc2)N1n1c(-c2ccccc2)nc2ccccc2c1=O
Example label: 0


#### Constructing a vocabulary

In [30]:
chars = set()
for string in train_data:
    chars.update(string)
all_sys = sorted(list(chars)) + ['<bos>', '<eos>', '<pad>', '<unk>']
vocab = all_sys
c2i = {c: i for i, c in enumerate(all_sys)}
i2c = {i: c for i, c in enumerate(all_sys)}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vector = torch.eye(len(c2i))

def char2id(char):
    if char not in c2i:
        return c2i['<unk>']
    else:
        return c2i[char]

def id2char(id):
    if id not in i2c:
        return i2c[32] 
    else:
        return i2c[id]

def string2ids(string,add_bos=False, add_eos=False):
    ids = [char2id(c) for c in string]
    if add_bos:
        ids = [c2i['<bos>']] + ids
    if add_eos:
        ids = ids + [c2i['<eos>']]
    return ids

def ids2string(ids, rem_bos=True, rem_eos=True):
    if len(ids) == 0:
        return ''
    if rem_bos and ids[0] == c2i['<bos>']:
        ids = ids[1:]
    if rem_eos and ids[-1] == c2i['<eos>']:
        ids = ids[:-1]
    string = ''.join([id2char(id) for id in ids])
    return string

def string2tensor(string, device='model'):
    ids = string2ids(string, add_bos=True, add_eos=True)
    tensor = torch.tensor(ids, dtype=torch.long, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    return tensor

tensor = [string2tensor(string, device=device) for string in train_data]
vector = torch.eye(len(c2i)) 

#### Model

In [31]:
# Parameters
q_bidir = True 
q_d_h = 256
q_n_layers = 1
q_dropout = 0.5

d_n_layers = 3
d_dropout = 0
d_z = 128
d_d_h = 512

In [32]:
class CVAE(nn.Module):
  def __init__(self, vocab, vector):
    super().__init__()
    self.vocabulary = vocab
    self.vector = vector
    self.n_classes = 2
    
    n_vocab, d_emb = len(vocab), vector.size(1) # 60, 60
    self.x_emb = nn.Embedding(n_vocab, d_emb, padding_idx = c2i['<pad>']) # 60, 60, 58
    self.x_emb.weight.data.copy_(vector) # 60, 60
  
    # Encoder
    self.encoder_rnn = nn.GRU(d_emb, q_d_h, num_layers=q_n_layers, batch_first=True, dropout=q_dropout if q_n_layers > 1 else 0, bidirectional=q_bidir) # 60, 256
    q_d_last = q_d_h * (2 if q_bidir else 1) # 512
    self.q_mu = nn.Linear(q_d_last + self.n_classes, d_z) # 514, 128
    self.q_logvar = nn.Linear(q_d_last + self.n_classes, d_z) # 514, 128
  
    # Decoder
    self.decoder_rnn = nn.GRU(d_emb + d_z, d_d_h + self.n_classes, num_layers=d_n_layers, batch_first=True, dropout=d_dropout if d_n_layers > 1 else 0) # 60 + 128, 514
    self.decoder_latent = nn.Linear(d_z, d_d_h) # 128, 512
    self.decoder_fullyc = nn.Linear(d_d_h + self.n_classes, n_vocab) # 514, 60
  
  
    # Grouping the model's parameters
    self.encoder = nn.ModuleList([self.encoder_rnn, self.q_mu, self.q_logvar])
    self.decoder = nn.ModuleList([self.decoder_rnn, self.decoder_latent, self.decoder_fullyc])
    self.vae = nn.ModuleList([self.x_emb, self.encoder, self.decoder])
    

  def device(self):
    return next(self.parameters()).device

  def string2tensor(self, string, label, device='model'):
    ids = string2ids(string, add_bos=True, add_eos=True)
    tensor = torch.tensor(ids, dtype=torch.long, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    return tensor, torch.tensor(label, dtype=torch.long, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")) #?
    
  def tensor2string(self, tensor):
    ids = tensor.tolist()
    string = ids2string(ids, rem_bos=True, rem_eos=True)
    return string
  
  def forward(self, x, y):
    z, kl_loss = self.forward_encoder(x, y)
    recon_loss = self.forward_decoder(x, z, y)
    return kl_loss, recon_loss
  
  def forward_encoder(self, x, y):
    x = [self.x_emb(i_x) for i_x in x]
    x = nn.utils.rnn.pack_sequence(x)
    y = torch.tensor(y, dtype=torch.long, device=device)
    y = torch.nn.functional.one_hot(y, self.n_classes).float() # 512, 2
    _, h = self.encoder_rnn(x, None) # 2, 512, 256
    h = h[-(1 + int(self.encoder_rnn.bidirectional)):] # 2, 512, 256
    h = torch.cat(h.split(1), dim=-1).squeeze(0) 
    h = torch.cat((h, y), dim = -1) # 512, 514 
    mu, logvar = self.q_mu(h), self.q_logvar(h) # 512, 128 
    eps = torch.randn_like(mu)
    z = mu + (logvar / 2).exp() * eps 
    kl_loss = 0.5 * (logvar.exp() + mu ** 2 - 1 - logvar).sum(1).mean()
    return z, kl_loss
  
  def forward_decoder(self, x, z, y):
    lengths = [len(i_x) for i_x in x]
    x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value= c2i['<pad>'])
    x_emb = self.x_emb(x) # 512, len, 60
    y = torch.tensor(y, dtype=torch.long, device=device)
    y = torch.nn.functional.one_hot(y, self.n_classes).float() 
    z_0 = z.unsqueeze(1).repeat(1, x_emb.size(1), 1) # 512, len, 128
    x_input = torch.cat([x_emb, z_0], dim=-1) # 512, len, 60 + 128
    x_input = nn.utils.rnn.pack_padded_sequence(x_input, lengths, batch_first=True)
    h_0 = self.decoder_latent(z) # 512, 512 
    h_0 = torch.cat((h_0, y), dim = -1) # 512, 514
    h_0 = h_0.unsqueeze(0).repeat(self.decoder_rnn.num_layers, 1, 1) # 3, 512, 512
    output, _ = self.decoder_rnn(x_input, h_0)
    output, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first=True) # 512, len, 512
    y = self.decoder_fullyc(output) # 512, len, 60
    recon_loss = F.cross_entropy(y[:, :-1].contiguous().view(-1, y.size(-1)),x[:, 1:].contiguous().view(-1),ignore_index= c2i['<pad>'])
    return recon_loss
  
    
  def sample_z_prior(self, n_batch):
    return torch.randn(n_batch,self.q_mu.out_features, device=self.x_emb.weight.device)
  
  def sample(self, n_batch, label, max_len=100, z=None, temp=1.0):
    with torch.no_grad():
      if z is None:
        z = self.sample_z_prior(n_batch) # 512, 128 
        z = z.to(device)
        z_0 = z.unsqueeze(1) # 512, 1, 128
        h = self.decoder_latent(z) # 512, 512 
        y = torch.tensor([label] * n_batch, dtype=torch.long, device=device)
        y = torch.nn.functional.one_hot(y, self.n_classes).float()
        h = torch.cat((h, y), dim = -1) # 512, 514 
        h = h.unsqueeze(0).repeat(self.decoder_rnn.num_layers, 1, 1) # 3, 512, 512
        w = torch.tensor(c2i['<bos>'], device=device).repeat(n_batch)
        x = torch.tensor([c2i['<pad>']], device=device).repeat(n_batch, max_len)
        x[:, 0] = c2i['<bos>']
        end_pads = torch.tensor([max_len], device=device).repeat(n_batch)
        eos_mask = torch.zeros(n_batch, dtype=torch.bool, device=device)

        for i in range(1, max_len):
          x_emb = self.x_emb(w).unsqueeze(1) # 512, 1, 60
          x_input = torch.cat([x_emb, z_0], dim=-1) # 512, 1, 60 + 128
          o, h = self.decoder_rnn(x_input, h)
          y = self.decoder_fullyc(o.squeeze(1))
          
          y = F.softmax(y / temp, dim=-1)
          y = torch.clamp(y, 1e-8, 1.0) 
          
          w = torch.multinomial(y, 1)[:, 0]
          x[~eos_mask, i] = w[~eos_mask]
          i_eos_mask = ~eos_mask & (w == c2i['<eos>'])
          end_pads[i_eos_mask] = i + 1
          eos_mask = eos_mask | i_eos_mask

        new_x = []
        for i in range(x.size(0)):
            new_x.append(x[i, :end_pads[i]])

      return [self.tensor2string(i_x) for i_x in new_x]

#### Training

In [33]:
# Parameters
n_last = 1000
n_batch = 512
kl_start = 0
kl_w_start = 0.0
kl_w_end = 1.0
n_epoch = 100
n_workers = 0

clip_grad  = 50
lr_start = 0.003
lr_n_period = 10
lr_n_mult = 1
lr_end = 3 * 1e-4
lr_n_restarts = 6

In [34]:
df_losses = pd.DataFrame(columns=['epoch', 'kl_weight', 'lr', 'kl_loss', 'recon_loss', 'loss'])

In [35]:
class KLAnnealer:
    def __init__(self,n_epoch):
        self.i_start = kl_start
        self.w_start = kl_w_start
        self.w_max = kl_w_end
        self.n_epoch = n_epoch

        self.inc = (self.w_max - self.w_start) / (self.n_epoch - self.i_start)

    def __call__(self, i):
        k = (i - self.i_start) if i >= self.i_start else 0
        return self.w_start + k * self.inc
      
      
class CosineAnnealingLRWithRestart(_LRScheduler):
    def __init__(self , optimizer):
        self.n_period = lr_n_period
        self.n_mult = lr_n_mult
        self.lr_end = lr_end

        self.current_epoch = 0
        self.t_end = self.n_period

        # Also calls first epoch
        super().__init__(optimizer, -1)

    def get_lr(self):
        return [self.lr_end + (base_lr - self.lr_end) *
                (1 + math.cos(math.pi * self.current_epoch / self.t_end)) / 2
                for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
        self.last_epoch = epoch
        self.current_epoch += 1

        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

        if self.current_epoch == self.t_end:
            self.current_epoch = 0
            self.t_end = self.n_mult * self.t_end         
            
            
class CircularBuffer:
    def __init__(self, size):
        self.max_size = size
        self.data = np.zeros(self.max_size)
        self.size = 0
        self.pointer = -1

    def add(self, element):
        self.size = min(self.size + 1, self.max_size)
        self.pointer = (self.pointer + 1) % self.max_size
        self.data[self.pointer] = element
        return element

    def last(self):
        assert self.pointer != -1, "Can't get an element from an empty buffer!"
        return self.data[self.pointer]

    def mean(self):
        return self.data.mean()
      
      
class Logger(UserList):
    def __init__(self, data=None):
        super().__init__()
        self.sdata = defaultdict(list)
        for step in (data or []):
            self.append(step)

    def __getitem__(self, key):
        if isinstance(key, int):
            return self.data[key]
        elif isinstance(key, slice):
            return Logger(self.data[key])
        else:
            ldata = self.sdata[key]
            if isinstance(ldata[0], dict):
                return Logger(ldata)
            else:
                return ldata

    def append(self, step_dict):
        super().append(step_dict)
        for k, v in step_dict.items():
            self.sdata[k].append(v)

In [36]:
def _n_epoch():
    return sum(lr_n_period * (lr_n_mult ** i) for i in range(lr_n_restarts))
  
def _train_epoch(model, epoch, train_loader, kl_weight, optimizer=None):
    if optimizer is None:
        model.eval()
    else:
        model.train()
      
    kl_loss_values = CircularBuffer(n_last)
    recon_loss_values = CircularBuffer(n_last)
    loss_values = CircularBuffer(n_last)

    for data in train_loader:
        input_batch, labels = [item[0] for item in data], [item[1] for item in data]
        input_batch = tuple(data.to(device) for data in input_batch)
        labels = [label.to(device) for label in labels]
      
    # forward
        kl_loss, recon_loss = model(input_batch, labels)
        loss = kl_weight * kl_loss + recon_loss
    
    # backward
        if optimizer is not None:
            optimizer.zero_grad()
            loss.backward()
            clip_grad_norm_(get_optim_params(model),clip_grad)
            optimizer.step()
      
        kl_loss_values.add(kl_loss.item())
        recon_loss_values.add(recon_loss.item())
        loss_values.add(loss.item())
        lr = (optimizer.param_groups[0]['lr'] if optimizer is not None else None)
      
    # update train_loader
        kl_loss_value = kl_loss_values.mean()
        recon_loss_value = recon_loss_values.mean()
        loss_value = loss_values.mean()
        postfix = [f'loss={loss_value:.5f}',f'(kl={kl_loss_value:.5f}',f'recon={recon_loss_value:.5f})',f'klw={kl_weight:.5f} lr={lr:.5f}']
    postfix = {'epoch': epoch,'kl_weight': kl_weight,'lr': lr,'kl_loss': kl_loss_value,'recon_loss': recon_loss_value,'loss': loss_value,'mode': 'Eval' if optimizer is None else 'Train'}
    return postfix
  
def _train(model, train_loader, val_loader=None, logger=None):
    optimizer = optim.Adam(get_optim_params(model), lr= lr_start)
    
    lr_annealer = CosineAnnealingLRWithRestart(optimizer)
    
    model.zero_grad()
    
    for epoch in tqdm(range(n_epoch), desc='Training', unit='epoch'):
      
        kl_annealer = KLAnnealer(n_epoch)
        kl_weight = kl_annealer(epoch)
        postfix = _train_epoch(model, epoch, train_loader, kl_weight, optimizer)
        df_losses.loc[len(df_losses.index)] = [postfix['epoch'], postfix['kl_weight'], postfix['lr'], postfix['kl_loss'], postfix['recon_loss'], postfix['loss']]
        lr_annealer.step()

def fit(model, train_data, train_label, val_data=None):
    logger = Logger() if False is not None else None
    train_loader = get_dataloader(model, train_data, train_label, shuffle=True)
    val_loader = None if val_data is None else get_dataloader(model, val_data, shuffle=False) # None
    _train(model, train_loader, val_loader, logger)
    return model

def get_collate_device(model):
    return model.device

def get_dataloader(model, train_data, train_label, collate_fn=None, shuffle=True):
    if collate_fn is None:
        collate_fn = get_collate_fn(model)
    return DataLoader(list(zip(train_data, train_label)), batch_size=n_batch, shuffle=shuffle, num_workers=n_workers, collate_fn=collate_fn)

def get_collate_fn(model):
    device = get_collate_device(model)

    def collate(train_data):
        train_data.sort(key=lambda x: len(x[0]), reverse=True) #?
        tensors = [model.string2tensor(string, label, device=device) for string, label in train_data]
        return tensors 

    return collate

def get_optim_params(model):
    return (p for p in model.parameters() if p.requires_grad)

#### Sample from model

In [37]:
# Parameters
n_samples = 1000
n_jobs = 1
max_len = 100
label = 1

In [38]:
class sample():
  def take_samples(model, n_batch, label):
    n = n_samples
    samples = []
    with tqdm(total=n_samples, desc='Generating samples') as T:
      while n > 0:
        current_samples = model.sample(min(n, n_batch), label, max_len)
        samples.extend(current_samples)
        n -= len(current_samples)
        T.update(len(current_samples))
    samples = pd.DataFrame(samples, columns=['SMILES'])
    return samples

#### Run

In [39]:
model = CVAE(vocab, vector).to(device)
fit(model, train_data, train_label)

Training: 100%|██████████| 100/100 [1:57:20<00:00, 70.40s/epoch]


CVAE(
  (x_emb): Embedding(60, 60, padding_idx=58)
  (encoder_rnn): GRU(60, 256, batch_first=True, bidirectional=True)
  (q_mu): Linear(in_features=514, out_features=128, bias=True)
  (q_logvar): Linear(in_features=514, out_features=128, bias=True)
  (decoder_rnn): GRU(188, 514, num_layers=3, batch_first=True)
  (decoder_latent): Linear(in_features=128, out_features=512, bias=True)
  (decoder_fullyc): Linear(in_features=514, out_features=60, bias=True)
  (encoder): ModuleList(
    (0): GRU(60, 256, batch_first=True, bidirectional=True)
    (1-2): 2 x Linear(in_features=514, out_features=128, bias=True)
  )
  (decoder): ModuleList(
    (0): GRU(188, 514, num_layers=3, batch_first=True)
    (1): Linear(in_features=128, out_features=512, bias=True)
    (2): Linear(in_features=514, out_features=60, bias=True)
  )
  (vae): ModuleList(
    (0): Embedding(60, 60, padding_idx=58)
    (1): ModuleList(
      (0): GRU(60, 256, batch_first=True, bidirectional=True)
      (1-2): 2 x Linear(in_featu

In [40]:
# Save model
torch.save(model.state_dict(), f'checkpoints/cvae_model_epoch{n_epoch}.pt')

In [41]:
# Load model state dictionary
model = CVAE(vocab, vector).to(device)
model.load_state_dict(torch.load(f'checkpoints/cvae_model_epoch{n_epoch}.pt'))

<All keys matched successfully>

In [42]:
# Sample SMILES
model.eval()
df_sample = sample.take_samples(model, n_batch, label)
df_sample

Generating samples: 100%|██████████| 1000/1000 [00:00<00:00, 1176.58it/s]


,SMILES
0,N#CC1=C(N)OC(c2ccc([N+](=O)[O-])cc2)O1
1,CC1(C)CC(=O)C2=C(C1)Nc1ccccc1NC1=NCC21
2,CCCCCCCCC1=C(O)C(=NCCN)C(C(=O)OC)C1N(Cc1ccccc1...
3,COc1ccc(C=Cc2ccc3cccnc3c2O)cc1
4,CSc1ncnc2c1cnn2CCCCCCCCO
...,...
995,Cc1occc1C(=S)Nc1ccc(Cl)c(C(=O)OC2CCCC2)c1
996,O=S(=O)(O)c1cc(NN=Cc2cc(S(=O)(=O)O)c3cccnc3c2O...
997,CC(NC(=O)C1CC(=O)NC(=O)C1C)C(=O)O
998,CCOC(=O)C=C1C(=O)Oc2cc(=O)cc(OC(C)=O)c21


In [43]:
# Save generated molecules
df_sample.to_csv(f'generated_molecules/cvae/cvae_epoch{n_epoch}.csv', index = False)

In [44]:
df_losses.to_csv(f'checkpoints/losses/cvae_epoch{n_epoch}.csv', index = False)